# PyBullet demo

This notebook demonstrates how to train a USA model on a PyBullet environment. The environment is taken from [here](https://github.com/adubredu/pybullet_kitchen).

To get started, first make sure to install some dependencies:

```bash
pip install pybullet scikit-learn scipy
```

In [ ]:
import pybullet as pb
import numpy as np
import imageio
from typing import Iterator
from torch import Tensor
from pathlib import Path
from IPython.display import Image

In [ ]:
def capture_frame() -> np.ndarray:
    # camera POV/FOV and image parameters, modify as necessary
    camTargetPos = [-5, 0, 1.477]
    yaw = 90
    pitch = -10.0
    roll = 0
    upAxisIndex = 2
    camDistance = 3
    nearPlane = 0.01
    farPlane = 100
    fov = 60
    pixelWidth = 500
    pixelHeight = 300

    viewMatrix = pb.computeViewMatrixFromYawPitchRoll(camTargetPos, camDistance, yaw, pitch, roll, upAxisIndex)
    aspect = pixelWidth / pixelHeight
    projectionMatrix = pb.computeProjectionMatrixFOV(fov, aspect, nearPlane, farPlane)

    img_arr = pb.getCameraImage(pixelWidth, pixelHeight, viewMatrix, projectionMatrix)
    w = img_arr[0]  # width of the image, in pixels
    h = img_arr[1]  # height of the image, in pixels
    rgb = img_arr[2]  # color data RGB
    dep = img_arr[3]  # depth data
    np_img_arr = np.reshape(rgb, (h, w, 4))
    frame = np_img_arr[:, :, :3]
    return frame


def capture_sim(steps_per_frame: int, total_steps: int) -> list[np.ndarray]:
    frames = []
    for i in range(total_steps):
        pb.stepSimulation()
        if i % steps_per_frame == 0:
            frames.append(capture_frame())
    return frames


def write_gif(frames: Iterator[np.ndarray | Tensor], out_file: str | Path, *, fps: int = 30) -> None:
    writer = imageio.get_writer(str(out_file), mode="I", fps=fps)
    for frame in frames:
        if isinstance(frame, Tensor):
            frame = frame.detach().cpu().numpy()
        writer.append_data(frame)
    writer.close()

In [ ]:
class Kitchen:
    def __init__(self):
        pb.setAdditionalSearchPath("04_pybullet_data")
        kitchen_path = "kitchen_part_right_gen_convex.urdf"
        useFixedBase = True
        pb.setGravity(0, 0, -9.81)
        self.floor = pb.loadURDF("floor.urdf", useFixedBase=useFixedBase)
        self.kitchen = pb.loadURDF(kitchen_path, [-5, 0, 1.477], useFixedBase=useFixedBase)
        self.table = pb.loadURDF(
            "table.urdf",
            [1.0, 0, 0],
            pb.getQuaternionFromEuler([0, 0, 1.57]),
            useFixedBase=useFixedBase,
        )
        self.drawer_to_joint_id = {
            1: 18,
            2: 22,
            3: 27,
            4: 31,
            5: 37,
            6: 40,
            7: 48,
            8: 53,
            9: 56,
            10: 58,
            11: 14,
        }
        self.drawer_to_joint_limits = {
            1: (0, 1.57),
            2: (-1.57, 0),
            3: (-1.57, 0),
            4: (0, 1.57),
            5: (0.0, 0.4),
            6: (0.0, 0.4),
            7: (0, 1.57),
            8: (-1.57, 0),
            9: (0.0, 0.4),
            10: (0.0, 0.4),
            11: (0, 1.57),
        }

    def open_drawer(self, drawer_id):
        joint_id = self.drawer_to_joint_id[drawer_id]
        open_angle = self.drawer_to_joint_limits[drawer_id][1]
        pb.setJointMotorControl2(
            bodyIndex=self.kitchen,
            jointIndex=joint_id,
            controlMode=pb.POSITION_CONTROL,
            targetPosition=open_angle,
            maxVelocity=0.5,
        )

    def close_drawer(self, drawer_id):
        joint_id = self.drawer_to_joint_id[drawer_id]
        close_angle = self.drawer_to_joint_limits[drawer_id][0]
        pb.setJointMotorControl2(
            bodyIndex=self.kitchen,
            jointIndex=joint_id,
            controlMode=pb.POSITION_CONTROL,
            targetPosition=close_angle,
            maxVelocity=0.5,
        )

In [ ]:
pb.connect(pb.DIRECT)
pb.resetSimulation()

pb.setGravity(0, 0, -9.81)
pb.setPhysicsEngineParameter(enableConeFriction=0)

total_steps_per_control = 1000
steps_per_frame = 100

frames = []
kitchen = Kitchen()

for i in range(10):
    drawer_id = i + 1
    kitchen.open_drawer(drawer_id)
    frames += capture_sim(steps_per_frame, total_steps_per_control)

for i in range(10):
    drawer_id = i + 1
    kitchen.close_drawer(drawer_id)
    frames += capture_sim(steps_per_frame, total_steps_per_control)

In [ ]:
write_gif(iter(frames), "video.gif")
Image("video.gif")